In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-corpus/text_corpus.txt


In [17]:
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM,Dense,Flatten,Embedding
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.corpus import stopwords
import string

import joblib

In [4]:
file=open("../input/text-corpus/text_corpus.txt",'r')
text=file.read()
file.close()
print(text[0:100])

A Game Of Thrones 
Book One of A Song of Ice and Fire 
By George R. R. Martin 
PROLOGUE 
"We should 


In [5]:
def clean_text(text):
    sample=text
    sample=re.sub('[%s]'% re.escape(string.punctuation),'',sample)
    sample=[word for word in sample.split() if word.isalpha()]
    sample=[word.lower() for word in sample]
    sample=' '.join(sample)
    return sample

In [6]:
cleaned_text=clean_text(text)
cleaned_text[0:100]
print(len(cleaned_text.split()))

292883


In [7]:
seq_doc=[]
seq_len=50
l=seq_len+1
tokens=[w for w in cleaned_text.split()]
for i in range(l,len(tokens)):
    seq=tokens[i-l:i]
    line=' '.join(seq)
    seq_doc.append(line)
    

In [8]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(seq_doc)
sequences=tokenizer.texts_to_sequences(seq_doc)
vocab_size=len(tokenizer.word_index)+1

In [9]:
sequences=np.array(sequences)
X,Y=sequences[:,:-1],sequences[:,-1]
Y=to_categorical(Y,num_classes=vocab_size)
seq_len=X.shape[1]

In [10]:
print(vocab_size,seq_len)

11924 50


In [11]:
def model(vocab_size,seq_length):
    model=Sequential()
    model.add(Embedding(vocab_size,50,input_length=seq_length))
    model.add(LSTM(100,return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [12]:
model=model(11924,50)

2022-11-13 15:19:44.754425: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            596200    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 11924)             1204324   
Total params: 1,951,424
Trainable params: 1,951,424
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X,Y,batch_size=128,epochs=5)

2022-11-13 15:20:23.933103: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
2288/2288 [==============================] - 529s 229ms/step - loss: 6.4792 - accuracy: 0.0688
Epoch 2/5
2288/2288 [==============================] - 526s 230ms/step - loss: 5.9337 - accuracy: 0.0954
Epoch 3/5
2288/2288 [==============================] - 523s 229ms/step - loss: 5.6369 - accuracy: 0.1141
Epoch 4/5
2288/2288 [==============================] - 515s 225ms/step - loss: 5.4261 - accuracy: 0.1238
Epoch 5/5
2288/2288 [==============================] - 523s 228ms/step - loss: 5.2674 - accuracy: 0.1306


In [21]:
model.save('text_gen_model.h5')
joblib.dump(tokenizer,open('tokenizer_text_gen.joblib','wb'))

In [30]:
def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
    result=list()
    in_text=seed_text
    for i in range(n_words):
        encoded=tokenizer.texts_to_sequences([in_text])[0]
        encoded=pad_sequences([encoded],maxlen=seq_length,truncating='pre')
        y=model.predict(encoded,verbose=0)
        ymax=np.argmax(y,axis=1)
        out_word=''
        for word,index in tokenizer.word_index.items():
            if index==ymax:
                out_word=word
                break
        in_text=in_text+' '+out_word
        result.append(out_word)
    return ' '.join(result)

In [31]:
seed_text=seq_doc[np.random.randint(0,len(seq_doc))]
print(seed_text+'\n')
print(generate_seq(model,tokenizer,seq_len,seed_text,50))

table he spoke softly yet the high officers of the nights watch all fell quiet the better to hear what the ancient had to say i think he is a giant come among us here at the end of the world tyrion answered gently ive been called many things my lord

the king said i am a man and the king said i am a man and the king said i am a man and the king said i am a man and the king said i am a man and the king said i am a man and the king
